<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pooling" data-toc-modified-id="Pooling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pooling</a></span></li><li><span><a href="#Convolution:-Feature-Transformation" data-toc-modified-id="Convolution:-Feature-Transformation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Convolution: Feature Transformation</a></span></li><li><span><a href="#Convolution:-Pattern-Finder" data-toc-modified-id="Convolution:-Pattern-Finder-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Convolution: Pattern Finder</a></span></li><li><span><a href="#Color-Image:-multidimensional-array" data-toc-modified-id="Color-Image:-multidimensional-array-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Color Image: multidimensional array</a></span></li><li><span><a href="#CIFAR-CNN" data-toc-modified-id="CIFAR-CNN-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>CIFAR CNN</a></span></li></ul></div>

In [ ]:
import os

import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Uncomment if you are using colab
# !mkdir ./data

# !wget https://raw.githubusercontent.com/shestakoff/sphere-ml-intro/master/2020/lecture11-cnn/data/house_processed.jpeg -O ./data/house_processed.jpeg
# !wget https://raw.githubusercontent.com/shestakoff/sphere-ml-intro/master/2020/lecture11-cnn/data/msu.jpg -O ./data/msu.jpg

In [ ]:
img = PIL.Image.open("./data/house_processed.jpeg").convert('L')
np_img = np.array(img).astype("float32")

image = np_img.reshape(1, np_img.shape[0], np_img.shape[1], 1)

In [ ]:
def show_image(image, figsize=(8,8)):
    plt.figure(figsize=figsize)
    plt.imshow(image, cmap="gray")
    plt.axis("off");
    plt.show()

In [ ]:
show_image(image[0, :, :, 0])

# Pooling

In [ ]:
pooled_image = image.copy()

for i in range(2):
    pooled_image = tf.nn.avg_pool2d(pooled_image, ksize=(1,2,2,1), strides=(1,2,2,1), padding="SAME")
    show_image(pooled_image[0], figsize=(8 // (2 ** i), 8 // (2 ** i)))

# Convolution: Feature Transformation

In [ ]:
filters =np.stack([
    np.array([
        [[0], [0], [0]],
        [[0], [1], [0]],
        [[0], [0], [0]]
    ]),
    np.array([
        [[1], [1], [1]],
        [[1], [1], [1]],
        [[1], [1], [1]]
    ]) / 9,
    np.array([
        [[1], [0], [-1]],
        [[2], [0], [-2]],
        [[1], [0], [-1]]
    ]),
    np.array([
        [[1], [2], [1]],
        [[0], [0], [0]],
        [[-1], [-2], [-1]]
    ])
],
    axis=-1
)

outputs = tf.nn.conv2d(image, filters, strides=1, padding="VALID")

for i in range(filters.shape[-1]):
    result = tf.abs(outputs[0, :, :, i])
    show_image(result)

In [ ]:
sobol = tf.sqrt(outputs[0, :, :, 2] ** 2 + outputs[0, :, :, 3] ** 2)
show_image(sobol)

# Convolution: Pattern Finder

In [ ]:
show_image(pooled_image[0,:,:,0])

filters = np.zeros(shape=(3, 3, 1, 2), dtype=np.float32)
filters[:, 1, :, 0] = 1 
filters[1, :, :, 1] = 1

outputs = tf.nn.conv2d(pooled_image, filters, strides=1, padding="VALID")

show_image(outputs[0, :, :, 0])
show_image(outputs[0, :, :, 1])

# Color Image: multidimensional array

In [ ]:
msu = PIL.Image.open("./data/msu.jpg")

In [ ]:
plt.figure(figsize=(12,5))
plt.imshow(msu)
plt.axis("off");

In [ ]:
np_msu = np.array(msu)

In [ ]:
np_msu.shape

In [ ]:
plt.figure(figsize=(60,60))
plt.subplot(131)
plt.imshow(np_msu[:,:,0], cmap="gray")
plt.axis("off")
plt.subplot(132)
plt.imshow(np_msu[:,:,1], cmap="gray")
plt.axis("off")
plt.subplot(133)
plt.imshow(np_msu[:,:,2], cmap="gray")
plt.axis("off")
plt.show()

# CIFAR CNN

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

In [ ]:
class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

N_ROWS = 5
N_COLS = 5

plt.figure(figsize=(10,10))
for i in range(N_ROWS * N_COLS):
    plt.subplot(N_ROWS, N_COLS, i + 1)
    plt.title(class_names[y_train[i]])
    plt.imshow(x_train[i])
    plt.axis("off")
plt.show()

In [ ]:
K = np.unique(y_train).size
print("number of classes:", K)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=[32, 32, 3]),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(K)
])

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"
)

history = model.fit(
    x_train, y_train, validation_data=(x_test, y_test),
    epochs=5, batch_size=32
)